In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [5]:
df = pd.read_csv('shortDataset.csv')

In [6]:
df.head()

,comment,label
0,welcome bake sir,irrelevant
1,bol diya ab bol diya ab kya kr sakte h,irrelevant
2,avi yaha yoh baj ne wale hai sir,irrelevant
3,all the very best,irrelevant
4,jai alak sir,irrelevant


In [7]:
df['label'].value_counts()

label
irrelevant    10000
feedbak       10000
doubt         10000
Name: count, dtype: int64

In [8]:
df['label'] = df['label'].replace({'irrelevant':'0', 'doubt':'1', 'feedbak':'2'}).astype(int)

In [9]:
comments = df['comment'].to_list()
labels = to_categorical(df['label'].to_list())

In [10]:
X_train, X_test, y_train, y_test = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding = 'post', truncating = 'post')
X_test = pad_sequences(X_test, maxlen=max_length, padding = 'post', truncating = 'post')


In [23]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Adjust output dimension based on your labels (binary or multi-class)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
750/750 [==============================] - 80s 105ms/step - loss: 1.0988 - accuracy: 0.3383 - val_loss: 1.0988 - val_accuracy: 0.3335
Epoch 2/10
750/750 [==============================] - 82s 109ms/step - loss: 92268437504.0000 - accuracy: 0.3326 - val_loss: 1.0989 - val_accuracy: 0.3327
Epoch 3/10
750/750 [==============================] - 82s 110ms/step - loss: 234392128.0000 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3343
Epoch 4/10
750/750 [==============================] - 85s 114ms/step - loss: 1169772544.0000 - accuracy: 0.3304 - val_loss: 1.0989 - val_accuracy: 0.3343
Epoch 5/10
750/750 [==============================] - 84s 113ms/step - loss: 1.0986 - accuracy: 0.3303 - val_loss: 1.0986 - val_accuracy: 0.3325
Epoch 6/10
750/750 [==============================] - 85s 113ms/step - loss: 1.0982 - accuracy: 0.3380 - val_loss: 1.0988 - val_accuracy: 0.3325
Epoch 7/10
750/750 [==============================] - 84s 112ms/step - loss: 4436091469824.0000 - accur

In [25]:
model.save("LSTM_V1")

INFO:tensorflow:Assets written to: LSTM_V1\assets


INFO:tensorflow:Assets written to: LSTM_V1\assets


In [27]:
test_comment = ['fuck']
test_comment = tokenizer.texts_to_sequences(test_comment)
test_comment = pad_sequences(test_comment, maxlen=max_length, padding = 'post', truncating = 'post')
print(test_comment)
pred_comment = model.predict(test_comment)
pred_comment = pred_comment.argmax()
mapping = {0:'irrelevant', 1:'doubt', 2:'feedbak'}
mapping[pred_comment]

[[6125    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
1/1 [==============================] - 0s 35ms/step


'doubt'